In [ ]:
%%capture

!pip install flair

In [ ]:
# Prepare data
data_path = "/content/drive/MyDrive/data/HindiData/"
exp_id = "HindiEXPnER2"
save_dir = "/content/drive/MyDrive/data/HindiData"+exp_id
model_name = 'xlm-roberta-base'
model_name = 'google/muril-base-cased'

lines = open(data_path+"data40.dis",'r').readlines()
labels = open(data_path+"data40.labels",'r').readlines()

In [ ]:
print(lines[:5])
print(labels[:5])

['तनु को एक उम्म्म एक टेक्स्ट मैसेज भेजो\n', 'यह जान को उउउ जॉन को भेज दो\n', 'अनिल को करो एक अम्म्म ईमेल करो\n', 'क्या राकेश को ईमेल भेज सकते हो\n', 'रोहम अरे रोहन को ईमेल में इमेज अटैच्ड करो सेंड करो\n']
['0 0 1 1 0 0 0 0\n', '0 1 1 1 0 0 0 0\n', '0 0 1 0 1 0 0\n', '0 0 0 0 0 0 0\n', '1 1 0 0 0 0 0 0 0 0 0\n']


In [ ]:
import random
temp = list(zip(lines, labels))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
lines, labels = list(res1), list(res2)

train_lines = lines[:int(0.8*len(lines))]
train_labels = labels[:int(0.8*len(labels))]
val_lines = lines[int(0.8*len(lines)):int(0.9*len(lines))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_lines = lines[int(0.9*len(lines)):]
test_labels = labels[int(0.9*len(labels)):]

In [ ]:
import pandas as pd
test = pd.DataFrame({'Sentence': test_lines, 'tag': test_labels })

In [ ]:
import pandas as pd
sentence_col = []
word_col = []
tag_col = []
for i in range(len(train_lines)):
  sent = train_lines[i][:-1]
  label = train_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_train = pd.DataFrame()
df_train['Sentence #'] = sentence_col
df_train['Word'] = word_col
df_train['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(val_lines)):
  sent = val_lines[i][:-1]
  label = val_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_val = pd.DataFrame()
df_val['Sentence #'] = sentence_col
df_val['Word'] = word_col
df_val['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(test_lines)):
  sent = test_lines[i][:-1]
  label = test_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_test = pd.DataFrame()
df_test['Sentence #'] = sentence_col
df_test['Word'] = word_col
df_test['Tag'] = tag_col

In [ ]:
df_train =df_train.fillna(method ="ffill")
df_val=df_val.fillna(method ="ffill")
df_test=df_test.fillna(method ="ffill")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df_train["Sentence #"] = LabelEncoder().fit_transform(df_train["Sentence #"] )
df_val["Sentence #"] = LabelEncoder().fit_transform(df_val["Sentence #"] )
df_test["Sentence #"] = LabelEncoder().fit_transform(df_test["Sentence #"] )

In [ ]:
df_train.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
df_val.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
df_test.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
df_train.head(20)

,sentence_id,words,labels
0,0,क्या,0
1,0,मेरी,0
2,0,एक,0
3,0,अह,1
4,0,अह,1
5,0,अह,1
6,0,नाचती,0
7,0,हुई,0
8,0,वीडियो,0
9,0,बना,0


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.0 MB

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args,use_cuda=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_dir="/content/drive/MyDrive/data/HindiData/"
from sklearn.metrics import accuracy_score

In [ ]:
model.train_model(df_train,df_val=df_test,output_dir=model_dir, acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/79 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/79 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/79 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/79 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/79 [00:00<?, ?it/s]

(395, 0.33271123295343374)

In [ ]:
result, model_outputs, preds_list = model.eval_model(df_test)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:159: UndefinedMetricWarning: F-s

In [ ]:
result

{'eval_loss': 0.3090132296085358,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [ ]:
prediction, model_output = model.predict(test.Sentence)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
list1 = [[list(dct.values())[0] for dct in inner_list] for inner_list in prediction]
p1_list = [[int(value) for value in inner_list] for inner_list in list1]

In [ ]:
a1_list = [[int(char) for char in string.strip() if char.isdigit()] for string in test_labels]

In [ ]:
result_list = [
    [
        [a_val, b_val] for a_val, b_val in zip(a_list, b_list)
    ] for a_list, b_list in zip(a1_list, p1_list)
]

result_list
# [actual label , predicted label]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
a_list = mlb.fit_transform(a1_list)
p_list = mlb.transform(p1_list)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
#print(classification_report(a_list , p_list, target_names=target_names))
print(classification_report(a_list, p_list, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00       316
     class 1       0.96      0.84      0.90       284

   micro avg       0.98      0.93      0.95       600
   macro avg       0.98      0.92      0.95       600
weighted avg       0.98      0.93      0.95       600
 samples avg       0.98      0.93      0.94       600



In [ ]:
accuracy_score(a_list, p_list)

0.8259493670886076